In [ ]:
import os

In [ ]:
cwd = os.getcwd()
xyz_dir = f"{cwd}/../XYZ"
MP2_dir = f"{cwd}/../MP2"
SAPT_dir = f"{cwd}/../SAPT"
ALMO_dir = f"{cwd}/../ALMO"

dimer_name = "ethanol_WD"


In [ ]:
# define index and factor

index = {
    "ethanol_ethanol": "001",
    "ethanol_WA": "002",
    "ethanol_WD": "003",
}

factor = [0.8, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2]

In [ ]:
# define functions

def read_arc(file_path):
    frames = []
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    i = 0
    while i < len(lines):
        # Read the first line of the frame, which contains the number of atoms
        try:
            num_atoms = int(lines[i].split()[0])
        except ValueError:
            i += 1
            continue  # Skip malformed lines
        
        frame_data = {
            "atoms": [],
            "coordinates": []
        }
        
        # Read the atomic data
        for j in range(i + 1, i + 1 + num_atoms):
            parts = lines[j].split()
            atom_symbol = parts[1]  # The second column contains the atomic symbol
            x, y, z = map(float, parts[2:5])  # Extract x, y, z coordinates
            
            frame_data["atoms"].append(atom_symbol)
            frame_data["coordinates"].append((x, y, z))
        
        frames.append(frame_data)
        i += num_atoms + 1  # Move to the next frame

    return frames

# Generate MP2 input files for each frame
def write_mp2_inputs(arcA, arcB, output_dir, dimer_name, index, factor):
    for idx, (frameA, frameB) in enumerate(zip(arcA, arcB)):
        output_file = f"{output_dir}/{index}_{dimer_name}_{factor[idx]:.2f}_mp2.inp"
        with open(output_file, "w") as file:
            file.write("MEMORY 10 GB\n\n")
            file.write("molecule dimer {\n")
            file.write("0 1\n")
            for atom, coord in zip(frameA["atoms"], frameA["coordinates"]):
                file.write(f"{atom:<2} {coord[0]:>10.6f} {coord[1]:>10.6f} {coord[2]:>10.6f}\n")
            file.write("--\n")
            file.write("0 1\n")
            for atom, coord in zip(frameB["atoms"], frameB["coordinates"]):
                file.write(f"{atom:<2} {coord[0]:>10.6f} {coord[1]:>10.6f} {coord[2]:>10.6f}\n")
            file.write("\nunits ang\nno_reorient\nno_com\nsymmetry c1\n}\n\n")
            file.write("set {\n")
            file.write("basis        aug-cc-pVTZ\n")
            file.write("scf_type     df\n")
            file.write("}\n\n")
            file.write("energy(\"mp2\", bsse_type=\"cp\")\n")

# Generate SAPT input files for each frame
def write_sapt_inputs(arcA, arcB, output_dir, dimer_name, index, factor):
    for idx, (frameA, frameB) in enumerate(zip(arcA, arcB)):
        output_file = f"{output_dir}/{index}_{dimer_name}_{factor[idx]:.2f}_sapt.inp"
        with open(output_file, "w") as file:
            file.write("MEMORY 10 GB\n\n")
            file.write("molecule dimer {\n")
            file.write("0 1\n")
            for atom, coord in zip(frameA["atoms"], frameA["coordinates"]):
                file.write(f"{atom:<2} {coord[0]:>10.6f} {coord[1]:>10.6f} {coord[2]:>10.6f}\n")
            file.write("--\n")
            file.write("0 1\n")
            for atom, coord in zip(frameB["atoms"], frameB["coordinates"]):
                file.write(f"{atom:<2} {coord[0]:>10.6f} {coord[1]:>10.6f} {coord[2]:>10.6f}\n")
            file.write("\nunits ang\nno_reorient\nno_com\nsymmetry c1\n}\n\n")
            file.write("set {\n")
            file.write("basis        aug-cc-pVTZ\n")
            file.write("scf_type     df\n")
            file.write("}\n\n")
            file.write("energy(\"sapt2+\")\n")

# Generate ALMO input files for each frame
def write_almo_inputs(arcA, arcB, output_dir, dimer_name, index, factor):
    for idx, (frameA, frameB) in enumerate(zip(arcA, arcB)):
        output_file = f"{output_dir}/{index}_{dimer_name}_{factor[idx]:.2f}_almo.inp"
        with open(output_file, "w") as file:
            file.write("$molecule\n")
            file.write("0 1\n--\n0 1\n")
            for atom, coord in zip(frameA["atoms"], frameA["coordinates"]):
                file.write(f"{atom:<2} {coord[0]:>10.6f} {coord[1]:>10.6f} {coord[2]:>10.6f}\n")
            file.write("--\n0 1\n")
            for atom, coord in zip(frameB["atoms"], frameB["coordinates"]):
                file.write(f"{atom:<2} {coord[0]:>10.6f} {coord[1]:>10.6f} {coord[2]:>10.6f}\n")
            file.write("$end\n\n")
            file.write("$rem\n")
            file.write("JOBTYPE          eda\n")
            file.write("EDA2             1\n")
            file.write("METHOD           wB97M-V\n")
            file.write("BASIS            def2-TZVPPD\n")
            file.write("SYMMETRY         false\n")
            file.write("MEM_TOTAL        50000\n")
            file.write("MEM_STATIC       10000\n")
            file.write("THRESH           14\n")
            file.write("SCF_CONVERGENCE  8\n")
            file.write("XC_GRID          000099000590\n")
            file.write("NL_GRID          1\n")
            file.write("EDA_CLS_DISP     true\n")
            file.write("FD_MAT_VEC_PROD  FALSE\n")
            file.write("$end\n")

# Example usage
fileA = f"{xyz_dir}/{dimer_name}_A.arc"
fileB = f"{xyz_dir}/{dimer_name}_B.arc"

arcA = read_arc(fileA)
arcB = read_arc(fileB)

write_mp2_inputs(arcA, arcB, MP2_dir, dimer_name, index[dimer_name], factor)
write_sapt_inputs(arcA, arcB, SAPT_dir, dimer_name, index[dimer_name], factor)
write_almo_inputs(arcA, arcB, ALMO_dir, dimer_name, index[dimer_name], factor)